**IMPORT WYMAGANYCH BIBLIOTEK I BAZ DANYCH**

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import itertools
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**IMPORT DANYCH - MAIL (spam, nie spam)**

In [2]:
spam_dataset = pd.read_csv('spam.csv', encoding = "ISO-8859-1", usecols=[0, 1], names=['Spam', 'Text'],
                           skiprows=1)
spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])

C:\Users\marci\AppData\Local\Temp\ipykernel_20500\1552169415.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])


**SERIA FUNKCJI i MODYFIKACJI MAIL:**

In [3]:
def remove_puncation(text):
    cleaned = ''.join([word for word in text if word not in string.punctuation])
    return cleaned

spam_dataset['Cleaned_Text'] = spam_dataset['Text'].apply(lambda x: remove_puncation(x))

In [4]:
def tokenize(text):
    clean_text = text.lower()
    tokenized_text = nltk.word_tokenize(clean_text)
    return tokenized_text

spam_dataset['Tokenized_Text'] = spam_dataset['Cleaned_Text'].apply(lambda x: tokenize(x))

In [5]:
stopwords = nltk.corpus.stopwords.words("english")

def remove_stopwords(text):
    without_stopwords = [word for word in text if word not in stopwords]
    return without_stopwords
spam_dataset['WithoutStop_Text'] = spam_dataset['Tokenized_Text'].apply(lambda x: remove_stopwords(x))

In [6]:
lemmater = nltk.WordNetLemmatizer()
def lemmatizing(text):
    lemmatized_words = [lemmater.lemmatize(word) for word in text]
    return lemmatized_words
spam_dataset['Lemmatized_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: lemmatizing(x))
spam_dataset

,Spam,Text,Cleaned_Text,Tokenized_Text,WithoutStop_Text,Lemmatized_Text
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, go, usf, life, around, though]"
...,...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, å£750, po...","[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,0,Will Ì_ b going to esplanade fr home?,Will Ì b going to esplanade fr home,"[will, ì, b, going, to, esplanade, fr, home]","[ì, b, going, esplanade, fr, home]","[ì, b, going, esplanade, fr, home]"
5569,0,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[pity, mood, soany, suggestion]"
5570,0,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[guy, bitching, acted, like, id, interested, b..."


**STWORZENIE BAZY DANYCH KTÓRA BĘDZIE CIĄGIEM SŁÓW - FORMAT WYMAGANY PRZEZ TfidfVectorizer**

In [7]:
spam_dataset['Chain_Text'] = spam_dataset['Lemmatized_Text'].apply(lambda x: ' '.join(x))

X = spam_dataset['Chain_Text']
y = spam_dataset['Spam']

**Dzielimy zbiór X i y na treningowy i testowy ze stratyfikacją według y w proporcji 80/20**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)

**Tworzymy pipeline - model_pipeline_tfidf_basic**

In [9]:
model_pipeline_tfidf_RF = Pipeline(steps=[
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range=(1, 1), max_df=0.5, min_df=0.001, use_idf=True)),
    ('classifier_rf_tfidf', RandomForestClassifier(max_depth=2, random_state=7))
])

model_pipeline_tfidf_RF.fit(X_train,y_train)

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer(max_df=0.5, min_df=0.001)),
                ('classifier_rf_tfidf',
                 RandomForestClassifier(max_depth=2, random_state=7))])

**Wyciągamy feature importance i dokonujemy selekcji cech - bierzemy tylko te których ważność jest większa niż 0.001**

In [10]:
vectorizer = model_pipeline_tfidf_RF.named_steps['vectorizer_tfidf']      # Wyciągamy obiekt TfidfVectorizer z naszego pipeline 
classifier = model_pipeline_tfidf_RF.named_steps['classifier_rf_tfidf']   # Wyciągamy obiekt RandomForestClassifier z naszego pipeline

feature_names = vectorizer.get_feature_names_out()             # wyjmujemy nazwy cech wygenerowanych przez TfidfVectorizer   
importances_features = classifier.feature_importances_         # wyjmujemy cechy i ich 'wagę'

mask = importances_features > 0.001                            # tworzymy tzn maskę gdzie: importances_features > 0.001    

X_train_tfidf = vectorizer.transform(X_train)            # poddajemy wektoryzacji zbiór X_train
X_train_filtered = X_train_tfidf[:, mask]                # filtrujemy X_train_tfidf wg "maski"

X_test_tfidf = vectorizer.transform(X_test)              # poddajemy wektoryzacji zbiór X_test
X_test_filtered = X_test_tfidf[:, mask]                  # filtrujemy X_test_tfidf wg "maski"

**Tworzymy nowy model ML (Random Forrest) z wyborem hiperparametrów korzystając z GridSearch**

In [11]:
RF_clf = RandomForestClassifier(max_depth=2, random_state=7)

In [12]:
params_RF_clf = {'max_depth':[1, 5, 10, 20, 30],
                 'n_estimators':[100, 200, 300, 500],
                 'min_samples_leaf': [1, 2, 5, 10]}

grid_search_RF_clf = GridSearchCV(RF_clf,
                                  params_RF_clf,
                                  cv=5,
                                  n_jobs=-1,
                                  verbose=10,
                                  scoring='accuracy'
)

grid_search_RF_clf.fit(X_train_filtered, y_train)

best_model_RF_clf = grid_search_RF_clf.best_estimator_


print("Najlepsze parametry dla grid_search_rf_tfidf:", grid_search_RF_clf.best_params_)
print("Najlepszy wynik (accuracy) dla grid_search_rf_tfidf na zbiorze walidacyjnym (kross validacja):", grid_search_RF_clf.best_score_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Najlepsze parametry dla grid_search_rf_tfidf: {'max_depth': 20, 'min_samples_leaf': 1, 'n_estimators': 100}
Najlepszy wynik (accuracy) dla grid_search_rf_tfidf na zbiorze walidacyjnym (kross validacja): 0.9661205981086398


In [13]:
best_model_RF_clf.fit(X_train_filtered, y_train)

RandomForestClassifier(max_depth=20, random_state=7)

In [14]:
score_model_pipeline_tfidf_RF = round(model_pipeline_tfidf_RF.score(X_test, y_test), 4)
score_best_model_RF_clf = round(best_model_RF_clf.score(X_test_filtered, y_test), 4)

In [15]:
print("***PODSUMOWANIE***\n")
print(f"📊 Pipeline składający się z wektoryzacji TFIDF i ML Random Forrest osiągnął wynik (score) na zbiorze testowym: {score_model_pipeline_tfidf_RF}.")

print(f"\n📊 ML Random Forrest szkolony na danych po selekcji cech (importances_features > 0.001) i przy użyciu GridSeachCV dla uzyskania optymalnych parametrów uzyskał wynik (score) na zbiorze testowym:: {score_best_model_RF_clf}.")
print("\nML Random Forrest szkolony na danych po selekcji cech i po GridSearchCV osiągną znacznie lepszy wynik!")

print("\n**Wnioski**")
print("\nDo trenowania modeli uczenia maszynowego na bazie danych tekstowych należy używać danych wektoryzowanych (np. przy pomocy TF-IDF), które są lepiej interpretowane przez modele ML. W przypadku modeli opartych na drzewach, takich jak Random Forest, możemy wykorzystać atrybut feature_importances_, aby zidentyfikować najbardziej istotne cechy w zbiorze danych. Ograniczenie liczby cech na podstawie ich ważności może znacząco przyspieszyć proces uczenia oraz poprawić skuteczność wyszukiwania optymalnych hiperparametrów. Należy pamiętać, że wektoryzacja jest przygotowaniem danych do nauki przez modele ML, więc należy sobie odpowiedzieć na pytanie czy dane są przygotowane prawidłowa oraz czy potrzebujemy dokonywać wektoryzacji każdorazowo.")

***PODSUMOWANIE***

📊 Pipeline składający się z wektoryzacji TFIDF i ML Random Forrest osiągnął wynik (score) na zbiorze testowym: 0.8664.

📊 ML Random Forrest szkolony na danych po selekcji cech (importances_features > 0.001) i przy użyciu GridSeachCV dla uzyskania optymalnych parametrów uzyskał wynik (score) na zbiorze testowym:: 0.9713.

ML Random Forrest szkolony na danych po selekcji cech i po GridSearchCV osiągną znacznie lepszy wynik!

**Wnioski**

Do trenowania modeli uczenia maszynowego na bazie danych tekstowych należy używać danych wektoryzowanych (np. przy pomocy TF-IDF), które są lepiej interpretowane przez modele ML. W przypadku modeli opartych na drzewach, takich jak Random Forest, możemy wykorzystać atrybut feature_importances_, aby zidentyfikować najbardziej istotne cechy w zbiorze danych. Ograniczenie liczby cech na podstawie ich ważności może znacząco przyspieszyć proces uczenia oraz poprawić skuteczność wyszukiwania optymalnych hiperparametrów. Należy pamiętać, że w